In [5]:
from scipy.optimize import minimize_scalar, minimize, LinearConstraint
import numpy as np



# Python3 SciPy

This is a shortened version of RealPython
Link: https://realpython.com/python-scipy-cluster-optimize/

When you need to optimize the input parameters for a function, scipy.optimize contains a number of useful methods for optimizing different kinds of functions:

* **minimize_scalar()** and **minimize()** to minimize a function of one variable and many variables, respectively
* **curve_fit()** to fit a function to a set of data
* **root_scalar()** and **root()** to find the zeros of a function of one variable and many variables, respectively
* **linprog()** to minimize a linear objective function with linear inequality and equality constraints


## Minimizing a Function With One Variable

A mathematical function that accepts one number and results in one output is called a scalar function. It’s usually
contrasted with multivariate functions that accept multiple numbers and also result in multiple numbers of output.

The scalar function is a quartic polynomial, and the objective is to find the minimum value of
the function.

The function is y = 3x⁴ - 2x + 1.

The function is plotted in the image below for a range of x from 0 to 1:

![one var chart](https://files.realpython.com/media/minimize_scalar.6858d6b6396f.png)

There’s a minimum value of this function at approximately x = 0.55. You can use minimize_scalar() to determine the
exact x and y coordinates of the minimum.

* First, import minimize_scalar() from scipy.optimize.
* Then, you need to define the objective function to be minimized:


In [3]:
def objective_function(x):
    return 3 * x ** 4 - 2 * x + 1




objective_function() takes the input x and applies the necessary mathematical operations to it, then returns the result.
In the function definition, you can use any mathematical functions you want. The only limit is that the function must
return a **single number at the end**.

Next, use minimize_scalar() to find the minimum value of this function. minimize_scalar() has only one required input,
which is the name of the objective function definition:


In [4]:
res = minimize_scalar(objective_function)
print(res)

     fun: 0.17451818777634331
    nfev: 16
     nit: 12
 success: True
       x: 0.5503212087491959


The optimum minimization value is x which is 0.55032120....

### Returns:

**OptimizeResult**
The optimization result represented as a OptimizeResult object. Important attributes are: x the solution array, success
a Boolean flag indicating if the optimizer exited successfully and message which describes the cause of the termination.
See OptimizeResult for a description of other attributes.

| Attribute | Description |
| --- | --- |
| fun | values of objective (jacobian and hessian) |
| nfev | number of evaluations of the objective and of its Jacobian and Hessian |
| nit | number of iteratiosn performed by the optimizer |
| success | T or F whether or not the optimizer exited successfully |
| x | the solution of the optimization |

## Minimizing a Function with Many Variables

scipy.optimize also includes the more general minimize(). This function can handle multivariate inputs and outputs and
has more complicated optimization algorithms to be able to handle this. In addition, minimize() can handle constraints
on the solution to your problem. You can specify three types of constraints:

1. **LinearConstraint**: The solution is constrained by taking the inner product of the solution x values with a
user-input array and comparing the result to a lower and upper bound.

2. **NonlinearConstraint**: The solution is constrained by applying a user-supplied function to the solution x values
and comparing the return value with a lower and upper bound.

3. **Bounds**: The solution x values are constrained to lie between a lower and upper bound.

When you use these constraints, it can limit the specific choice of optimization method that you’re able to use, since
not all of the available methods support constraints in this way.

### DEMO

How to use minimize().

Imagine you’re a stockbroker who’s interested in maximizing the total income from the sale of a fixed number of your
stocks. You have identified a particular set of buyers, and for each buyer, you know the price they’ll pay and how
much cash they have on hand.

You can phrase this problem as a **constrained optimization problem**. The objective function is that you want to
maximize your income. However, minimize() finds the minimum value of a function, so you’ll need to multiply your
objective function by -1 to find the x-values that produce the largest negative number.

There is one constraint on the problem, which is that the sum of the total shares purchased by the buyers does not
exceed the number of shares you have on hand. There are also bounds on each of the solution variables because each
buyer has an upper bound of cash available, and a lower bound of zero. Negative solution x-values mean that you’d
be paying the buyers!

In [6]:
# set a market of 10 buyers who’ll be buying 15 shares in total from you.
n_buyers = 10
n_shares = 15

Create arrays to store

* the price that each buyer pays
* the maximum amount they can afford to spend
* the maximum number of shares each buyer can afford, given the first two arrays.

Use random number generation in np.random to generate the arrays:


In [7]:
np.random.seed(10)
prices = np.random.random(n_buyers)
money_available = np.random.randint(1, 4, n_buyers)

In this code, you set the seed for NumPy’s random number generators. This function makes sure that each time you run
this code, you’ll get the same set of random numbers.

In line 7, you generate the array of prices the buyers will pay. np.random.random() creates an array of random numbers
on the half-open interval [0, 1). The number of elements in the array is determined by the value of the argument,
which in this case is the number of buyers.

In line 8, you generate an array of integers on the half-open interval from [1, 4), again with the size of the number
of buyers. This array represents the total cash each buyer has available. Now, you need to compute the maximum number
of shares each buyer can purchase:



In [8]:
n_shares_per_buyer = money_available / prices
print(prices, money_available, n_shares_per_buyer, sep="\n")

[0.77132064 0.02075195 0.63364823 0.74880388 0.49850701 0.22479665
 0.19806286 0.76053071 0.16911084 0.08833981]
[1 1 1 3 1 3 3 2 1 1]
[ 1.29647768 48.18824404  1.57816269  4.00638948  2.00598984 13.34539487
 15.14670609  2.62974258  5.91328161 11.3199242 ]


In line 9, you take the ratio of the money_available with prices to determine the maximum number of
shares each buyer can purchase.

The first row is the array of prices, which are floating-point numbers between 0 and 1. This row is followed by the
maximum cash available in integers from 1 to 4. Finally, you see the number of shares each buyer can purchase.

Create the constraints and bounds for the solver. The constraint is that the sum of the total purchased shares can’t
exceed the total number of shares available. This is a constraint rather than a bound because it involves more than
one of the solution variables.

To represent this mathematically, you could say that
```x[0] + x[1] + ... + x[n] = n_shares```
where n is the total number of buyers. You could take the dot or inner product of a vector of ones
with the solution values, and constrain that to be equal to n_shares. Remember that LinearConstraint takes the dot
product of the input array with the solution values and compares it to the lower and upper bound. You can use this
to set up the constraint on n_shares:


In [9]:
constraint = LinearConstraint(np.ones(n_buyers), lb=n_shares, ub=n_shares)


Create an array of ones with the length n_buyers and pass it as the first argument to LinearConstraint. Since
LinearConstraint takes the dot product of the solution vector with this argument, it’ll result in the sum of the
purchased shares.

This result is then constrained to lie between the other two arguments:

* The lower bound lb
* The upper bound ub

Since lb = ub = n_shares, this is an equality constraint because the sum of the values must be equal to both lb and ub.
If lb were different from ub, then it would be an inequality constraint.

Next, create the bounds for the solution variable. The bounds limit the number of shares purchased to be 0 on the
lower side and n_shares_per_buyer on the upper side. The format that minimize() expects for the bounds is a sequence
of tuples of lower and upper bounds:

In [10]:
bounds = [(0, n) for n in n_shares_per_buyer]

n this code, you use a comprehension to generate a list of tuples for each buyer. The last step before you run
the optimization is to define the objective function. Recall that you’re trying to maximize your income.
Equivalently, you want to make the negative of your income as large a negative number as possible.

The income that you generate from each sale is the price that the buyer pays multiplied by the number of shares
they’re buying. Mathematically, you could write this as

```prices[0]*x[0] + prices[1]*x[1] + ... + prices[n]*x[n]```

where n is again the total number of buyers.

Once again, you can represent this more succinctly with the inner product, or x.dot(prices). This means that your
objective function should take the current solution values x and the array of prices as arguments:


In [11]:
def objective_function(x, prices):
    return -x.dot(prices)

In this code, you define objective_function() to take two arguments. Then you take the dot product of x with
prices and return the negative of that value. Remember that you have to return the negative because you’re trying
to make that number as small as possible, or as close to negative infinity as possible. Finally, you
can call minimize():


In [16]:
res = minimize(
    objective_function,
    x0=10 * np.random.random(n_buyers),
    args=(prices,),
    constraints=constraint,
    bounds=bounds,
)
print(res)

     fun: -8.783020157087684
     jac: array([-0.7713207 , -0.02075195, -0.63364828, -0.74880397, -0.49850702,
       -0.22479665, -0.1980629 , -0.76053071, -0.16911077, -0.08833981])
 message: 'Optimization terminated successfully'
    nfev: 176
     nit: 16
    njev: 16
  status: 0
 success: True
       x: array([1.29647768e+00, 1.75304187e-15, 1.57816269e+00, 4.00638948e+00,
       2.00598984e+00, 3.48323773e+00, 5.44009282e-15, 2.62974258e+00,
       6.36407929e-15, 1.67391737e-15])


The call to minimize(), you pass five arguments:

1. objective_function: The first positional argument must be the function that you’re optimizing.

2. x0: The next argument is an initial guess for the values of the solution. In this case, you’re just providing
a random array of values between 0 and 10, with the length of n_buyers. For some algorithms or some problems,
choosing an appropriate initial guess may be important. However, for this example, it doesn’t seem too important.

3. args: The next argument is a tuple of other arguments that are necessary to be passed into the objective function.
minimize() will always pass the current value of the solution x into the objective function, so this argument serves
as a place to collect any other input necessary. In this example, you need to pass prices to objective_function(),
so that goes here.

4. constraints: The next argument is a sequence of constraints on the problem. You’re passing the constraint you
generated earlier on the number of available shares.

5. bounds: The last argument is the sequence of bounds on the solution variables that you generated earlier.

## Output

You can see message and status indicating the final state of the optimization. For this optimizer, a status of 0
means the optimization terminated successfully, which you can also see in the message. Since the optimization was
successful, fun shows the value of the objective function at the optimized solution values. You’ll make an
income of $8.78 from this sale.

You can see the values of x that optimize the function in res.x. In this case, the result is that you should sell
about 1.3 shares to the first buyer, zero to the second buyer, 1.6 to the third buyer, 4.0 to the fourth, and so on.

You should also check and make sure that the constraints and bounds that you set are satisfied. You
can do this with the following code:

In [17]:
print("The total number of shares is:", sum(res.x))
print("Leftover money for each buyer:", money_available - res.x * prices)

The total number of shares is: 15.0
Leftover money for each buyer: [8.88178420e-16 1.00000000e+00 4.32986980e-15 7.99360578e-15
 2.77555756e-15 2.21697984e+00 3.00000000e+00 8.65973959e-15
 1.00000000e+00 1.00000000e+00]
